In [1]:
import pandas as pd
import numpy as np

In [2]:
#import 2014
data_all_2014=pd.read_csv('Medicare_Data/2014MedicareProviderPayment.txt', delimiter='\t')

/Users/aurelperianu/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#fct rename columns lower letters; remove spaces
def lower_columns(pdx):
    cols = pdx.columns.tolist()
    cols = [col.lower() for col in cols]
    pdx.columns = cols
    pdx.rename(columns=lambda x: x.strip().replace(" ", "_"), inplace=True)
    return pdx
data_all_2014=lower_columns(data_all_2014)

In [4]:

#cleaup first row
data_all_2014= data_all_2014[data_all_2014.npi != 1]
#add report year
data_all_2014['report_year']='2014'

#add zip code
data_all_2014['nppes_provider_zip']=data_all_2014['nppes_provider_zip'].astype(str)
data_all_2014['nppes_provider_zip_base']=data_all_2014['nppes_provider_zip'].str.slice(0,5)
#data_all['nppes_provider_zip_base'].unique()

In [5]:
#data_all_2014.columns

In [6]:
#select WA state providers
data_all_2014['nppes_provider_state']=data_all_2014['nppes_provider_state'].astype(str)
wa_data_provider_14=data_all_2014[data_all_2014['nppes_provider_state']=='WA']

In [7]:
#exploratory
wa_data_provider_14.shape
#data_provider_wa.info()

(186239, 28)

In [8]:
#remove all non-individual provider from data
wa_data_provider_14=wa_data_provider_14[wa_data_provider_14.nppes_provider_first_name.notnull()]

In [9]:
#remove all non-medicare providers and the payments for drugs only
wa_data_provider_14=wa_data_provider_14[wa_data_provider_14['medicare_participation_indicator']=='Y']
wa_data_provider_14=wa_data_provider_14[wa_data_provider_14.hcpcs_drug_indicator !='Y']

In [10]:
#function to drop columns
drop_col=['nppes_entity_code','medicare_participation_indicator', 'hcpcs_drug_indicator']
def drop_columns(col_remove,pdx):
    columns=list(pdx.columns.values)
    for col in col_remove:
        if col in columns:
            pdx.drop(col, axis=1, inplace=True)
    return pdx
#drop_columns(drop_col,data_provider_wa)       

In [11]:
# data_provider_wa_2014['test_x']='100'
# data_provider_wa_2014['test_y']='100'
# drop_col=['test_x','test_y']

In [12]:
wa_data_provider_14=drop_columns(drop_col,wa_data_provider_14)   

In [13]:
#data2=data1[data1.nppes_provider_first_name.isnull()]
#data_provider_wa_2014['participation_indicator'].unique()
#data_provider_wa_2014['bene_day_srvc_cnt'].value_counts()

In [14]:
#fct remane columns
def name_columns(value_old,value_new,pdx):
    cols = pdx.columns.tolist()
    cols = [col.replace(value_old,value_new) for col in cols]
    pdx.columns = cols
    return pdx



In [15]:
wa_data_provider_14=name_columns("nppes_","", wa_data_provider_14)
wa_data_provider_14=name_columns("provider_","", wa_data_provider_14)
wa_data_provider_14=name_columns("medicare_","", wa_data_provider_14)
#data_provider_wa_2014=name_columns("medicare_","", data_provider_wa_2014)
wa_data_provider_14=name_columns("average_","", wa_data_provider_14)

#replace NaN with '' for all columns
wa_data_provider_14.fillna('',inplace=True)


In [16]:
#Calculate delta for payments
wa_data_provider_14['submitted_delta']=wa_data_provider_14['submitted_chrg_amt']-wa_data_provider_14['payment_amt']
wa_data_provider_14['allowed_delta']=wa_data_provider_14['allowed_amt']-wa_data_provider_14['payment_amt']


In [17]:
#wa_data_provider_14.info()

In [18]:
#wa_data_provider_14.to_pickle('wa_data_provider_14_pickle.pkl') 

In [19]:
#wa_data_provider_14= pd.read_pickle('wa_data_provider_14_pickle.pkl')

In [20]:
#wa_data_provider_14.head().T

In [21]:
#hcpcs_drug_indicator=N
# REMOVE line_srvc_cnt, standard_amt
#difference between submitted_chrg_amt;payment_amt
#difference between allowed_amt;payment_amt

In [22]:
# hosp_rating_gr=hosp_all.groupby(['provider_id','hospital_name','city','state','zip_code','county_name'])[['row_rating']].sum()
# hosp_rating_gr=hosp_rating_gr.reset_index()

In [23]:
#group by numeric values
# exclude ,'place_of_service' field
col_groupby=['npi','last_org_name','first_name','mi','credentials','gender','street1','street2','city','zip','state',
 'country','type','report_year','zip_base']
col_sum=['line_srvc_cnt','bene_unique_cnt','bene_day_srvc_cnt','allowed_amt','submitted_chrg_amt','payment_amt',
 'standard_amt','submitted_delta','allowed_delta']
#wa_data_provider_14_gr=wa_data_provider_14.groupby(col_groupby)[col_sum].sum()
wa_data_provider_14_gr=wa_data_provider_14.groupby(col_groupby)[col_sum].sum()
wa_data_provider_14_gr=wa_data_provider_14_gr.reset_index()

In [24]:
#wa_data_provider_14_gr.columns.tolist()

In [25]:
#wa_data_provider_14_gr.head()

In [26]:
#check duplicates - no duplicates
#wa_data_provider_14_gr[wa_data_provider_14_gr.duplicated(['npi'],keep=False)]

In [27]:
wa_data_provider_14_gr['submitted_delta_perc']=wa_data_provider_14_gr['submitted_delta']/wa_data_provider_14_gr['payment_amt']
wa_data_provider_14_gr['allowed_delta_perc']=wa_data_provider_14_gr['allowed_delta']/wa_data_provider_14_gr['payment_amt']
#x[x['npi']==1992980213].T

In [28]:
#wa_data_provider_14_gr.head().T

In [29]:
wa_data_provider_14_gr.to_pickle('wa_data_provider_14_gr_pickle.pkl') 

In [30]:
wa_data_provider_14_gr= pd.read_pickle('wa_data_provider_14_gr_pickle.pkl')

In [31]:
#data_2014.columns.tolist()

In [32]:
# #remove all columns that are not common in all files


# #set columns as sets to compare them
# # these can be put in a function
# set_col_2012=set(data_2012.columns)
# set_col_2013=set(data_2013.columns)
# set_col_2014=set(data_2014.columns)
# #print set_col_2014
# results_intersection = set.intersection(set_col_2012,set_col_2013,set_col_2014)

# #print results_intersection
# diff_2012=set_col_2012-results_intersection
# diff_2013=set_col_2013-results_intersection
# diff_2014=set_col_2014-results_intersection

# #print diff_2014

# results_union = list(set().union(diff_2012,diff_2013,diff_2014))

# # #print results_union

# data_2012=drop_columns(results_union,data_2012)  
# data_2013=drop_columns(results_union,data_2013)  
# data_2014=drop_columns(results_union,data_2014) 